In [1]:
import numpy as np
from math import gcd
from itertools import permutations
from itertools import combinations
from scipy.optimize import linprog
import matplotlib.pyplot as pp

class word(object):
    """ A word in F_2 
        
        w=b^{beta_n}a^{beta_n}...b^{beta_1}a^{beta_1}
        
        Attributes:
            a_chunks: List of largst contiguous 'a' string
            b_chunks: List of largst contiguous 'b' string

            alpha: array containing exponents of a 
            beta: array containing exponents of b

    """

    def __init__(self, string):
        
        self.string = string
        self.b_chunks = string.split('a')
        self.a_chunks = string.split('b')
        self.h_a = string.count('a')
        self.h_b = string.count('b')
        
    def beta(self):
        exponents = np.array([len(chunk) for chunk in self.b_chunks],dtype='int32') #Ignore zero length chunks
        return np.flip(exponents[exponents>0],0) #Return flipped since word is applied right-to-left
    
    def alpha(self):
        exponents = np.array([len(chunk) for chunk in self.a_chunks],dtype='int32') #Ignore zero length chunks
        return np.flip(exponents[exponents>0],0) #Return flipped since word is applied right-to-left

    def __str__(self):
        return self.string
####################################################################    
    
def partitions(n):  
    """
    Adapted from https://arxiv.org/abs/0909.2331
    According to the author, this is one of the fastest and efficient algo for printing partition of a number    
    """
    a = [0 for i in range(n + 1)]
    k = 1
    a[1] = n
    while k != 0:
        x = a[k - 1] + 1
        y = a[k] - 1
        k -= 1
        while x <= y:
            a[k] = x
            y -= x
            k += 1
        a[k] = x + y
        yield a[:k + 1]
####################################################################

def fancy_partitions(n,k,min=None,max=None):  
    """
    Produces a list of partition of n into k parts
    each part has value between min and max
    Each partition is a list object of size k that sums to n
    
    Output always shows in ascending order
    """
    
    if min==None:
        min=np.ones(k,dtype=int)
    if max==None:
        max=n*np.ones(k,dtype=int)
    
#     print(f'\nWe are partitioning {n} into {k} parts, each of which is at least {min} and at most {max}.'
#           +'\nPermutations are ALLOWED as long as each part satisfies the min-max condition.')
    
    if n==sum(max):
        yield max #The fringe case
        return
    
    for partition in partitions(n):
        if (len(partition)==k):
            for solution in set(permutations(partition)):
                if all(min[i]<=solution[i]<=max[i] for i in range(k)):
#                     print(solution)
                    yield solution

    
####################################################################    
            
def setup_lp(w, p, q, c=None, d=None):
    
    if ((c==None) & (d==None)):
        c = (p*w.h_a + q*w.h_b)
        d = q    
#         print('\nWe are in the fringe Case. Assuming R(w,p/q,1-).\n')
    g=np.dtype(int)
    g = gcd(c,d)
    c=c/g
    d=d/g
    
    n = len(w.alpha())
    
    k = n*d
    
    total = c*q - d*(w.h_a)*p - k 
#     print('\n c is:',c, '\n d is:',d, '\n total is:', total, '\n k is:', k, '\n n is:',n)
    return (int(c) , int(d), int(total), int(k), int(n))


####################################################################

def staircase(w,p,q,c=None,d=None):
    
    
# print('alpha exponent array is:',w.alpha(), '\nbeta exponent array is:', w.beta())
    
    c,d,total,k,n = setup_lp(w, p, q, c, d)


    for l_array in fancy_partitions(total,k,max=[q*(w.beta()[i%n])-1 for i in range(k)]):
        
        returnvalue=1 #Since u is at most 1

        s_array=np.empty(k, dtype='int32')

        for i in range(k):
#             print(sum(np.take(w.alpha(),range(i+1), mode='wrap'))*p + (i+1))
#             print(sum(l_array[:i]))
            s_array[i]=sum(np.take(w.alpha(),range(i+1), mode='wrap'))*p + (i+1) + sum(l_array[:i])

#         print('Current partition i.e. l-array is:', l_array)
#         print('s-array is:', s_array)

        A = np.zeros([k, q], dtype=int)

        for i in range(k):
            for j in range(s_array[i]+1,s_array[i]+l_array[i]+1):
                A[i,j%q - 1 ] +=1




        A_lastcol=np.array([[- w.beta()[i%n] for i in range(k)]],dtype=int)

        A = np.concatenate((A, A_lastcol.T), axis=1)

#         print(A)

        A_ub = A
        b_ub = np.zeros(k, dtype=int)

        A_eq = [np.append(np.ones(q, dtype=int),0)]
        b_eq = np.array([1])

        clin = np.append(np.zeros(q, dtype=int),1)

        res = linprog(clin, A_ub, b_ub, A_eq, b_eq, bounds=(0, 1))

#         print('Optimal value:', res.fun, '\nOptimal Solution Vector:', res.x, '\nCurrent Status:', res.status)

        if res.fun<=returnvalue:
            returnvalue=res.fun
    
    return (1-returnvalue)



        
w = word('baaabba')
qMax = 50

dataX = open(f'dataX_{w}.txt','w+')  

"""Either keep the data points for  future use or plot it"""

# xvalues=[]
# yvalues=[]

for q in range(1,qMax):
    for p in range(1,q):
        if gcd(p,q)==1:
#             xvalues.append(p/q)
#             yvalues.append(staircase(w,p,q))
#             print(f'Fringe length fr_{w}({p}/{q}) is: {staircase(w,p,q):.3f}')
            dataX.write("%0.6f , %0.6f \n" % (p/q, staircase(w,p,q)))
            

# pp.figure(figsize=(10,10))

# pp.axis(xmin=0,xmax=1,ymin=0,ymax=1)    
    
# pp.vlines(xvalues,[0], yvalues, linewidth=1e-1)
    
        


